#  Movie recommendation system

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [3]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [5]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [6]:
movies.shape

(10329, 3)

In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [8]:
movies.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [9]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [10]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
10324,146684,Cosmic Scrat-tastrophe (2015),Animation|Children|Comedy
10325,146878,Le Grand Restaurant (1966),Comedy
10326,148238,A Very Murray Christmas (2015),Comedy
10327,148626,The Big Short (2015),Drama


In [11]:
ratings

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
...,...,...,...,...
105334,668,142488,4.0,1451535844
105335,668,142507,3.5,1451535889
105336,668,143385,4.0,1446388585
105337,668,144976,2.5,1448656898


In [12]:
df=pd.merge(movies,ratings,on='movieId')
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286


'''1. Create a popularity-based recommender system at a genre level. The user will input a genre (g),
minimum rating threshold (t) for a movie, and no. of
recommendations(N) for which it should be recommended top N movies which are most popular within that 
genre (g) ordered by ratings in descending order
where each movie has at least (t) reviews.'''

In [13]:
# genres=[]
# for genre in movies.genres:
    
#     x=genre.split('|')
#     for i in x:
#          if i not in genres:
#             genres.append(str(i))
# genres=str(genres)    
# movie_title=[]
# for title in movies.title:
#     movie_title.append(title[0:-7])
# movie_title=str(movie_title)    

In [14]:
trend=pd.DataFrame(df.groupby('title')['rating'].mean())
trend['total number of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count()) 


In [15]:
trend.sort_values('total number of ratings',ascending=False).head(10)

,rating,total number of ratings
title,,
Pulp Fiction (1994),4.160000,325
Forrest Gump (1994),4.138264,311
"Shawshank Redemption, The (1994)",4.454545,308
Jurassic Park (1993),3.659864,294
"Silence of the Lambs, The (1991)",4.194828,290
Star Wars: Episode IV - A New Hope (1977),4.188645,273
"Matrix, The (1999)",4.264368,261
Terminator 2: Judgment Day (1991),3.960474,253
Braveheart (1995),4.108871,248


### Create a content-based recommender system that recommends top N movies based on similar movie(m) genres

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
print(df["title"].head(5))

0    Toy Story (1995)
1    Toy Story (1995)
2    Toy Story (1995)
3    Toy Story (1995)
4    Toy Story (1995)
Name: title, dtype: object


In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
df["title"] = df["title"].fillna("")

tfidf_matrix = tfidf.fit_transform(df["title"])
print(tfidf_matrix.shape)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim.shape)

indices = pd.Series(movies_df.index, index=df["title"]).drop_duplicates()
print(indices.head())

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    """
    in this function,
        we take the cosine score of given movie
        sort them based on cosine score (movie_id, cosine_score)
        take the next 10 values because the first entry is itself
        get those movie indices
        map those indices to titles
        return title list
    """
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    # (a, b) where a is id of movie, b is sim_score

    movies_indices = [ind[0] for ind in sim_scores]
    movies = df["title"].iloc[movies_indices]
    return movies

In [18]:
df['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'A Very Murray Christmas (2015)', 'The Big Short (2015)',
       'Marco Polo: One Hundred Eyes (2015)'], dtype=object)

In [ ]:
print("################ Content Based Filtering - plot#############")
print()
print("Recommendation for Toy Story")
print(get_recommendations("Toy Story"))
print()
print("Recommendations for Toy Story")
print(get_recommendations("Toy Story"))

### Create a collaborative based recommender system which recommends top N movies based on “K” similar users for a target user “u”

#### ● Import libraries and load dataset
●Exploratory Data Analysis including:

a)Understanding of distribution of the features available

b)Finding unique users and movies

c)Average rating and Total movies at genre level.

d)Unique genres considered..

●Design the 3 different types of recommendation modules as mentioned in the objectives.

●Additional/Optional: Create a GUI interface using Python libraries (ipywidgets etc.) to play around with the recommendation modules.

In [18]:
import numpy as np 
import pandas as pd

In [19]:
# Import libraries and load dataset
df=pd.read_csv('ratings.csv')
df1=pd.read_csv("movies.csv")

In [20]:
final_df=pd.merge(df1,df,on='movieId')
final_df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,5.0,859046895
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,1303501039
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,5.0,858610933
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.0,850815810
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14,4.0,851766286


In [21]:
# unique user and movie
final_df['userId'].unique()

array([  2,   5,   8,  11,  14,  17,  28,  29,  30,  31,  32,  38,  44,
        45,  50,  52,  54,  60,  62,  69,  70,  71,  72,  74,  81,  83,
        84,  86,  88,  89,  93,  96,  97, 100, 103, 105, 107, 108, 109,
       113, 114, 116, 117, 118, 120, 122, 128, 129, 130, 133, 134, 137,
       140, 141, 147, 151, 156, 158, 159, 164, 165, 166, 168, 169, 171,
       176, 177, 179, 180, 187, 188, 192, 195, 196, 198, 202, 203, 213,
       215, 218, 220, 224, 226, 230, 232, 233, 235, 236, 239, 244, 250,
       255, 262, 264, 268, 271, 272, 278, 282, 283, 286, 289, 290, 296,
       299, 303, 308, 310, 311, 314, 325, 328, 329, 331, 335, 336, 337,
       339, 347, 349, 350, 352, 354, 358, 365, 374, 376, 378, 379, 380,
       383, 386, 387, 393, 399, 402, 405, 409, 410, 413, 419, 421, 422,
       423, 425, 426, 432, 435, 437, 439, 440, 441, 449, 450, 451, 453,
       454, 455, 458, 460, 461, 462, 469, 471, 481, 484, 485, 490, 493,
       495, 496, 501, 506, 511, 517, 528, 530, 531, 532, 534, 53

In [22]:
final_df['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       ..., 'A Very Murray Christmas (2015)', 'The Big Short (2015)',
       'Marco Polo: One Hundred Eyes (2015)'], dtype=object)

In [23]:
# Average rating
final_df['rating'].mean()

3.5168503593161127

In [24]:
# Total movie at generes level
final_df.groupby('title')['genres'].value_counts().sum()

105339

In [25]:
# unique genres 
final_df['genres'].unique()

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance',
       'Comedy|Drama|Romance', 'Comedy', 'Action|Crime|Thriller',
       'Adventure|Children', 'Action', 'Action|Adventure|Thriller',
       'Comedy|Horror', 'Adventure|Animation|Children', 'Drama',
       'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance',
       'Action|Comedy|Crime|Drama|Thriller', 'Comedy|Crime|Thriller',
       'Crime|Drama|Horror|Mystery|Thriller', 'Drama|Sci-Fi',
       'Children|Drama', 'Adventure|Drama|Fantasy|Mystery|Sci-Fi',
       'Mystery|Sci-Fi|Thriller', 'Adventure|Romance|IMAX', 'Drama|War',
       'Action|Crime|Drama', 'Action|Adventure|Fantasy',
       'Comedy|Drama|Thriller', 'Mystery|Thriller',
       'Animation|Children|Drama|Musical|Romance',
       'Crime|Mystery|Thriller', 'Adventure|Drama', 'Children|Comedy',
       'Drama|Mystery', 'Drama|Thriller', 'Comedy|Crime',
       'Action|Sci-Fi|Thriller', 'Action|Comedy|Horror|Thriller'

In [ ]:
# Understanding of distribution of the features available
import matplotlib.pyplot as plt 
import seaborn as sns
sns.histplot(final_df,y='genres',x='title',bins=50,hue='rating',kde=True)